In [1]:
import sklearn
import sklearn.datasets
import pandas as pd
from sklearn import linear_model
import operator
import numpy as np
import itertools
from sklearn import metrics
from scipy.optimize import minimize 
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression

from fangorn.files_prep import get_data, data_to_pandas
from fangorn.preprocessing import splitting, feature_selection
from fangorn.training import classifiers

from category_encoders import OneHotEncoder
import pymit

import ray
ray.init(num_cpus=20)

2020-09-13 10:32:19,294	WARNING worker.py:1373 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2020-09-13 10:32:19,296	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-09-13_10-32-19_296032_109325/logs.
2020-09-13 10:32:19,408	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:10449 to respond...


2020-09-13 10:32:19,532	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:13518 to respond...
2020-09-13 10:32:19,534	INFO services.py:809 -- Starting Redis shard with 10.0 GB max memory.
2020-09-13 10:32:19,552	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-09-13_10-32-19_296032_109325/logs.
2020-09-13 10:32:19,562	WARNING services.py:1301 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_memory` when calling ray.init() or ray start.
2020-09-13 10:32:19,579	INFO services.py:1475 -- Starting the Plasma object store with 20.0 GB memory using /dev/shm.
2020-09-13 10:32:19,594	WARNING services.py:912 -- Failed to start the reporter. The reporter requires 'pip install psutil'.


{'node_ip_address': '10.96.22.5',
 'redis_address': '10.96.22.5:10449',
 'object_store_address': '/tmp/ray/session_2020-09-13_10-32-19_296032_109325/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-09-13_10-32-19_296032_109325/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-09-13_10-32-19_296032_109325'}

In [2]:
# read dataset
all_datasets = get_data.get_all_data(only='ml_challenge')
this_dataset = 'madeline'

# configure dataset
X_all, y_all = data_to_pandas.read_prepare_data(this_dataset)
# X_all = feature_selection.extra_trees_feature_selection(X_all, y_all)
dataset_split_dict = splitting.simple_train_test_val_split(X_all, y_all)

X_train = dataset_split_dict['train']['X']
y_train = dataset_split_dict['train']['y']
X_test = dataset_split_dict['test']['X']
y_test = dataset_split_dict['test']['y']

All ML_CHALLENGE files ready!


In [3]:
def numpy_discretize(X_train, X_test, max_gran=10):
    """
    multi-granularity discretization
    method. The basic idea is simple: instead of using a fine-tuned
    granularity, we discretize each numerical feature into several, rather
    than only one, categorical features, each with a different granularity.
    
    min granularity = 3
    
    Sometimes de edge values did not permit to execute correct discretization
    if this happens the step is not executed
    """
    
    # separa dados numericos que precisam de binarizacao
    is_numeric = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numeric_features = X_train.select_dtypes(include=is_numeric)
    discrete_features = []
    print(f"Discretizing {len(numeric_features.columns)} features...")
    feat_count = 0
    for feat in numeric_features:
        if feat_count % 50 == 0:
            print(f" Working in {feat}")
        X_train_np = X_train[[feat]].to_numpy()
        X_test_np = X_test[[feat]].to_numpy()
        for gran in range(3, max_gran+1):
            try:
                D_train = np.zeros([X_train.shape[0], 1])
                D_test = np.zeros([X_test.shape[0], 1])
                # calc numpy histogram and apply to features
                hist, bin_edges = np.histogram(X_train_np[:, 0], bins=gran)
                D_train[:, 0] = np.digitize(X_train_np[:,0], bin_edges, right=False)
                D_test[:, 0] = np.digitize(X_test_np[:,0], bin_edges, right=False)

                # apply back to pandas
                X_train[f"{feat}_{gran}"] = D_train
                X_test[f"{feat}_{gran}"] = D_test
            except:
                print(f"Not possible to correct work on cut {feat} > {gran}")
                break
        
        feat_count += 1
        X_train = X_train.drop(feat, axis=1)
        X_test = X_test.drop(feat, axis=1)
        
    return X_train, X_test

In [4]:
X_train_discrete, X_test_discrete = numpy_discretize(X_train.copy(), X_test.copy())

Discretizing 259 features...
 Working in 0
 Working in 50
 Working in 100
 Working in 150
 Working in 200
 Working in 250


In [5]:
def hjmi_selector(X, y, bins, max_features):
    
    X = X.to_numpy()
    Y = y.to_numpy().ravel()
    bins = 10

    [tmp, features] = X.shape
    D = np.zeros([tmp, features])

    for i in range(features):
        N, E = np.histogram(X[:,i], bins=bins)
        D[:,i] = np.digitize(X[:,i], E, right=False)

    selected_features = []
    j_h = 0
    hjmi = None

    for i in range(0,max_features):
        JMI = np.zeros([features], dtype=np.float)
        for X_k in range(features):
            if X_k in selected_features:
                continue
            jmi_1 = pymit.I(D[:,X_k], Y, bins=[bins,2])
            jmi_2 = 0
            for X_j in selected_features:
                tmp1 = pymit.I(D[:,X_k], D[:,X_j], bins=[bins,bins])
                tmp2 = pymit.I_cond(D[:,X_k], D[:,X_j], Y, bins=[bins,bins,2])
                jmi_2 += tmp1 - tmp2
            if len(selected_features) == 0:
                JMI[X_k] += j_h + jmi_1
            else:
                JMI[X_k] += j_h + jmi_1 - jmi_2/len(selected_features)
        
        f = JMI.argmax()
        j_h = JMI[f]
        if (hjmi == None) or ((j_h - hjmi)/hjmi > 0.03):
            r = 0
            if hjmi != None:
                r = ((j_h - hjmi)/hjmi) 

            hjmi = j_h
            selected_features.append(f)
            print("{:0>3d} {:>3d} {} - {}".format(len(selected_features), f, j_h, r))
        else:
            return selected_features

In [6]:
selected_features = hjmi_selector(X_train_discrete.copy(), y_train.copy(), bins=10, max_features=300)

001 1902 0.05330720180283708 - 0
002 687 0.16296850394591655 - 2.0571573527470948
003 1479 0.24124708922089674 - 0.48032953226936453
004 1535 0.3256292677095313 - 0.3497749082119306
005 1303 0.40522943912027903 - 0.24445029763649154
006 1903 0.47796290479927034 - 0.17948712175721954
007 686 0.5569395618224394 - 0.16523595498762994
008 423 0.6293285495907539 - 0.12997637936051878
009 567 0.7025048318410445 - 0.11627675607260533
010 1534 0.7778947246681412 - 0.10731583529400575
011 684 0.8504461209951012 - 0.0932663431519107
012 1901 0.9213396957940156 - 0.08336045405905593
013 1471 0.9905527228656312 - 0.07512215894699666
014 685 1.059457156533011 - 0.06956160139365622
015 1533 1.1295119070972939 - 0.06612325013078524
016 1087 1.1961512331162913 - 0.058998338663159657
017 1487 1.2633314029794234 - 0.05616360875046699
018 1900 1.3317803462321318 - 0.054181304360264776
019 1863 1.3988168468659294 - 0.05033600384887572
020 1470 1.4668449383850863 - 0.048632593803523944
021 1531 1.535341983

In [15]:
selected_features = [1902, 687, 1479, 1535, 1303, 1903, 686, 423, 567, 1534, 684, 1901, 1471, 685, 1533, 1087, 1487, 1900, 1863, 1470, 1531, 1486, 1478, 671, 1523, 1302, 1431, 1583, 2022]

In [18]:
filtered_train = X_train_discrete[X_train_discrete.columns[selected_features]]
filtered_test = X_test_discrete[X_test_discrete.columns[selected_features]]

In [10]:
filtered_train = X_train_discrete.copy()
filtered_test = X_test_discrete.copy()

In [28]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(final_X_train, y_train)


/home/fernando.favoretti/anaconda3/envs/basic/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


KNeighborsClassifier(n_neighbors=3)

In [26]:
def get_dummies(X_train, X_test):
    ohe = OneHotEncoder(cols=X_train.columns).fit(X=X_train)
    X_train = ohe.transform(X_train)
    X_test = ohe.transform(X_test)
    return X_train, X_test

final_X_train, final_X_test = get_dummies(filtered_train.copy(), filtered_test.copy())


In [27]:
final_X_train

,237_9_1,237_9_2,237_9_3,237_9_4,237_9_5,237_9_6,237_9_7,237_9_8,237_9_9,237_9_10,...,252_9_1,252_9_2,252_9_3,252_9_4,252_9_5,252_9_6,252_9_7,252_9_8,252_9_9,252_9_10
2953,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1560,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2098,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
71,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1636,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2680,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1854,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
217,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2164,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [31]:
def run_initial_logit(X_train, y_train, X_test, y_test):
    clf = LogisticRegression().fit(X_train, y_train[y_train.columns[0]])
    all_preds = clf.predict_proba(X_test)[:, 1]
    logloss = metrics.log_loss(y_test[y_test.columns[0]], all_preds)
    fpr, tpr, thresholds = metrics.roc_curve(y_test[y_test.columns[0]], all_preds, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    return clf, logloss, auc

start_classifier, logloss, auc = run_initial_logit(final_X_train.copy(), y_train.copy(), final_X_test.copy(), y_test.copy())
print(f"LOGLOSS: {logloss} - AUC: {auc}")

LOGLOSS: 0.6593040135656834 - AUC: 0.6934654471544716


/home/fernando.favoretti/anaconda3/envs/basic/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [15]:

def to_iterator(obj_ids):
    """
    # https://github.com/ray-project/ray/issues/5554
    """
    while obj_ids:
        done, obj_ids = ray.wait(obj_ids)
        yield ray.get(done[0])

    
def score_one_pair_parallel(combined_features, bsum, y_train):

    def _calc_logloss(y_true, preds):
        return metrics.log_loss(y_true[y_true.columns[0]], preds['preds'])

    def _obj(x, 
             y_train,
             combined_features,
             bsum):

        # add x to bsum
        this_w = np.array(x).reshape(-1,1)
        this_value = np.array(combined_features).reshape(1,-1)
        bsum_with_new_feature = bsum + np.matmul(this_w.T, this_value)
        this_preds = 1/(1 + np.exp(-bsum_with_new_feature)) 
        preds = pd.DataFrame(this_preds.reshape(-1,1), columns=['preds'])
        return _calc_logloss(y_train, preds)
    
    
    result = minimize(_obj, 1, args=(y_train,
                             combined_features['feature_value'],
                             bsum))

    this_coef = result['x'][0]
    this_logloss = result['fun']
    dict_result_combination = {"coef":this_coef,
                               "logloss" : this_logloss,
                               "feature": combined_features['feature_name']}

    return dict_result_combination


def iter_one_level(X_train, y_train, X_test, y_test, coef_dict, intercept):
    
    def _combine_all_features(current_training_set, pairwise_cols):
        all_features_combined = []
        for feature_pair in pairwise_cols:
            feature_name = str(feature_pair)
            if feature_name not in current_training_set.columns:
                combined_dict = {}
                combined_features = current_training_set[feature_pair[0]] | current_training_set[feature_pair[1]]
                combined_dict['feature_name'] = str(feature_pair)
                combined_dict['feature_value'] = combined_features
                if combined_features.equals(current_training_set[feature_pair[0]]) or combined_features.equals(current_training_set[feature_pair[1]]):
                    continue
                else:
                    all_features_combined.append(combined_dict)
        return all_features_combined
    
    def _chunker(seq, size):
        return (seq[pos:pos + size] for pos in range(0, len(seq), size))
    
    all_columns = list(X_train)
    pairwise_cols = list(itertools.combinations(all_columns, 2))
    all_results = {}
    col_coefs = np.array(list(coef_dict.values())).reshape(1,-1)
    bsum = np.add(np.matmul(col_coefs, X_train.values.T), intercept)
    
    print("Combinando features do nivel...")
    all_features_combined = _combine_all_features(X_train, pairwise_cols)
    print(f"{len(all_features_combined)} pares criados")
    
    all_results = []
    
    print("Iniciando paralelismo do nivel...")
    chunk = 0
    chuncksize = int(10e3)
    for group in _chunker(all_features_combined, chuncksize):
        print(f"\t working in chunk {chunk}")
        score_one_pair_parallel_ray = ray.remote(score_one_pair_parallel)
        results = [score_one_pair_parallel_ray.remote(ray.put(one_pair), bsum, ray.put(y_train)) for one_pair in group]
        for x in tqdm(to_iterator(results), total=len(results)):
            pass

        all_results.append(ray.get(results))
        chunk += chuncksize
        
    return sum(all_results, [])

def predict_logit(coef_dict, intercept, X_test, y_test):
    col_coefs = np.array(list(coef_dict.values())).reshape(1,-1)
    bsum = np.add(np.matmul(col_coefs, X_test.values.T), intercept)
    this_preds = 1/(1 + np.exp(-bsum)) 
    preds = pd.DataFrame(this_preds.reshape(-1,1), columns=['preds'])
    logloss = metrics.log_loss(y_test[y_test.columns[0]], preds['preds'])
    fpr, tpr, thresholds = metrics.roc_curve(y_test[y_test.columns[0]],  preds['preds'], pos_label=1)
    auc = metrics.auc(fpr, tpr)
    return logloss, auc

def beam_search(X_train, y_train, X_test, y_test):
    
    def _choose_best_feature(level_results):
        min_logloss = 9999
        bsf_feature = None
        bst_coef = None
        for dict_feature in level_results:
            if  dict_feature['logloss'] < min_logloss:
                min_logloss = dict_feature['logloss']
                bsf_feature = dict_feature['feature']
                bst_coef = dict_feature['coef']
        print(f"Level - choose {bsf_feature} -{bst_coef} -{min_logloss}")
        return bsf_feature, bst_coef, min_logloss
    
    current_training_set = X_train.copy()
    current_test_set = X_test.copy()
    start_classifier, start_logloss, start_auc = run_initial_logit(current_training_set, y_train, current_test_set, y_test)
    
    coef_dict = dict(list(zip(current_training_set.columns,start_classifier.coef_[0])))
    intercept = start_classifier.intercept_[0]
    
    print(f"Start logloss : {start_logloss} - Start AUC {start_auc}")
    last_logloss = start_logloss
    this_logloss = -np.inf
    accepted_features = []
    while this_logloss <= last_logloss:
        # eval one level
        level_results = iter_one_level(current_training_set, y_train, current_test_set, y_test, coef_dict, intercept)
        bst_feature, this_coef, _ = _choose_best_feature(level_results)
        
        # update X_train an X_test
        current_training_set[str(bst_feature)] = current_training_set[str(eval(bst_feature)[0])] | current_training_set[str(eval(bst_feature)[1])]
        current_test_set[str(bst_feature)] = current_test_set[str(eval(bst_feature)[0])] | current_test_set[str(eval(bst_feature)[1])]
        
        # retrain logit with new feature
#         this_clf, this_logloss, this_auc = run_initial_logit(current_training_set, y_train, current_test_set, y_test)
#         coef_dict = dict(list(zip(current_training_set.columns,this_clf.coef_[0])))
#         intercept = start_classifier.intercept_[0]
        coef_dict[bst_feature] = this_coef
        
        this_logloss, this_auc = predict_logit(coef_dict, intercept, current_test_set, y_test)
        print(f" new  logloss: {this_logloss} - new auc {this_auc}")
        
        coef_dict[bst_feature] = this_logloss
    
        current_logloss_diff = last_logloss - this_logloss
        
        if this_logloss > last_logloss:
            current_training_set =  current_training_set.drop([str(bst_feature)], axis=1)
            current_test_set =  current_test_set.drop([str(bst_feature)], axis=1)
            coef_dict.pop(str(bst_feature), None)
            
        else:
            last_logloss = this_logloss
            
        print(f"logloss gain with {bst_feature}: {current_logloss_diff}")
    return current_training_set, current_test_set, coef_dict, intercept   

In [ ]:
complete_X_train, complete_X_test, coef_dict, intercept = beam_search(final_X_train, y_train, final_X_test, y_test)

/home/fernando.favoretti/anaconda3/envs/basic/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Start logloss : 1.6233913150147528 - Start AUC 0.6328353658536585
Combinando features do nivel...
